In [1]:
# Import Modules
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [2]:
# Read json into df
df = pd.read_json("../original_data/country.json")
df.head()

,geometry,id,properties,type
0,"{'type': 'Polygon', 'coordinates': [[[61.21081...",AFG,{'name': 'Afghanistan'},Feature
1,"{'type': 'MultiPolygon', 'coordinates': [[[[16...",AGO,{'name': 'Angola'},Feature
2,"{'type': 'Polygon', 'coordinates': [[[20.59024...",ALB,{'name': 'Albania'},Feature
3,"{'type': 'Polygon', 'coordinates': [[[51.57951...",ARE,{'name': 'United Arab Emirates'},Feature
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-6...",ARG,{'name': 'Argentina'},Feature


In [3]:
# pull out the country name and put it into its own column
df["country"] = df.properties.map(lambda x: x["name"])
df.head()

,geometry,id,properties,type,country
0,"{'type': 'Polygon', 'coordinates': [[[61.21081...",AFG,{'name': 'Afghanistan'},Feature,Afghanistan
1,"{'type': 'MultiPolygon', 'coordinates': [[[[16...",AGO,{'name': 'Angola'},Feature,Angola
2,"{'type': 'Polygon', 'coordinates': [[[20.59024...",ALB,{'name': 'Albania'},Feature,Albania
3,"{'type': 'Polygon', 'coordinates': [[[51.57951...",ARE,{'name': 'United Arab Emirates'},Feature,United Arab Emirates
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-6...",ARG,{'name': 'Argentina'},Feature,Argentina


In [4]:
# read in wine data
df2 = pd.read_csv('../db/wine_data.csv')
df2.head()

,wine_id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,year
0,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0
1,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013.0
2,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013.0
3,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012.0
4,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011.0


In [5]:
# remove colums that won't aggregate
df2 = df2[['country', 'points', 'price']]
df2.head()

,country,points,price
0,Portugal,87,15.0
1,US,87,14.0
2,US,87,13.0
3,US,87,65.0
4,Spain,87,15.0


In [6]:
# using a dict and regex to rename some countries
dict1 = {
"US": "United States of America",
"England": "United Kingdom",
"Serbia": "Republic of Serbia"
}

# replacing the values
df2['country'] = df2['country'].replace(dict1)
df2.head()

,country,points,price
0,Portugal,87,15.0
1,United States of America,87,14.0
2,United States of America,87,13.0
3,United States of America,87,65.0
4,Spain,87,15.0


In [7]:
#create groupby object
sort_df = df2.groupby(['country'])

In [8]:
# agg avg rating
avg_score = sort_df['points'].mean()
avg_score.head()

country
Argentina                 86.760598
Armenia                   87.500000
Australia                 88.596642
Austria                   90.210774
Bosnia and Herzegovina    86.500000
Name: points, dtype: float64

In [9]:
# agg total wines
total_wines = sort_df['country'].count()
total_wines

country
Argentina                    3680
Armenia                         2
Australia                    2204
Austria                      2766
Bosnia and Herzegovina          2
Brazil                         38
Bulgaria                      139
Canada                        225
Chile                        4358
China                           1
Croatia                        70
Cyprus                         10
Czech Republic                 11
France                      16367
Georgia                        82
Germany                      2104
Greece                        440
Hungary                       142
India                           9
Israel                        488
Italy                       16083
Lebanon                        35
Luxembourg                      3
Macedonia                      12
Mexico                         70
Moldova                        57
Morocco                        28
New Zealand                  1364
Peru                           16
Portug

In [10]:
#create new df with this data
df3 = pd.DataFrame({
    "avg_score": avg_score,
    "total_wines": total_wines
})
df3.reset_index(level=0, inplace=True)
df3.head()

,country,avg_score,total_wines
0,Argentina,86.760598,3680
1,Armenia,87.500000,2
2,Australia,88.596642,2204
3,Austria,90.210774,2766
4,Bosnia and Herzegovina,86.500000,2


In [11]:
df3.nunique()

country        42
avg_score      39
total_wines    38
dtype: int64

In [12]:
df3.country.unique

<bound method Series.unique of 0                    Argentina
1                      Armenia
2                    Australia
3                      Austria
4       Bosnia and Herzegovina
5                       Brazil
6                     Bulgaria
7                       Canada
8                        Chile
9                        China
10                     Croatia
11                      Cyprus
12              Czech Republic
13                      France
14                     Georgia
15                     Germany
16                      Greece
17                     Hungary
18                       India
19                      Israel
20                       Italy
21                     Lebanon
22                  Luxembourg
23                   Macedonia
24                      Mexico
25                     Moldova
26                     Morocco
27                 New Zealand
28                        Peru
29                    Portugal
30          Republic of Serbia
31      

In [13]:
# merge wine data and json_normalize data
merge_df = df.merge(df3, how = 'inner', on= 'country')
merge_df.head()

,geometry,id,properties,type,country,avg_score,total_wines
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-6...",ARG,{'name': 'Argentina'},Feature,Argentina,86.760598,3680
1,"{'type': 'Polygon', 'coordinates': [[[43.58274...",ARM,{'name': 'Armenia'},Feature,Armenia,87.500000,2
2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",AUS,{'name': 'Australia'},Feature,Australia,88.596642,2204
3,"{'type': 'Polygon', 'coordinates': [[[16.97966...",AUT,{'name': 'Austria'},Feature,Austria,90.210774,2766
4,"{'type': 'Polygon', 'coordinates': [[[22.65715...",BGR,{'name': 'Bulgaria'},Feature,Bulgaria,87.928058,139


In [14]:
# define a function to add the properties back into a dictionary
def add_fields(row):
    properties = row.properties
    return properties.update({
        'country': row.country,
        'avg_score': row.avg_score,
        'total_wines': row.total_wines
    })

In [15]:
# use the function and add to the dataframe
merge_df["merged"] = merge_df.apply(add_fields, axis=1)
merge_df.head()

,geometry,id,properties,type,country,avg_score,total_wines,merged
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-6...",ARG,"{'name': 'Argentina', 'country': 'Argentina', ...",Feature,Argentina,86.760598,3680,None
1,"{'type': 'Polygon', 'coordinates': [[[43.58274...",ARM,"{'name': 'Armenia', 'country': 'Armenia', 'avg...",Feature,Armenia,87.500000,2,None
2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",AUS,"{'name': 'Australia', 'country': 'Australia', ...",Feature,Australia,88.596642,2204,None
3,"{'type': 'Polygon', 'coordinates': [[[16.97966...",AUT,"{'name': 'Austria', 'country': 'Austria', 'avg...",Feature,Austria,90.210774,2766,None
4,"{'type': 'Polygon', 'coordinates': [[[22.65715...",BGR,"{'name': 'Bulgaria', 'country': 'Bulgaria', 'a...",Feature,Bulgaria,87.928058,139,None


In [17]:
# verify the amount of countries
merge_df.country.nunique()

42

In [18]:
# only use the columns we are interested in
merge_df = merge_df[["type", "id", "properties", "geometry"]]
merge_df.head()

,type,id,properties,geometry
0,Feature,ARG,"{'name': 'Argentina', 'country': 'Argentina', ...","{'type': 'MultiPolygon', 'coordinates': [[[[-6..."
1,Feature,ARM,"{'name': 'Armenia', 'country': 'Armenia', 'avg...","{'type': 'Polygon', 'coordinates': [[[43.58274..."
2,Feature,AUS,"{'name': 'Australia', 'country': 'Australia', ...","{'type': 'MultiPolygon', 'coordinates': [[[[14..."
3,Feature,AUT,"{'name': 'Austria', 'country': 'Austria', 'avg...","{'type': 'Polygon', 'coordinates': [[[16.97966..."
4,Feature,BGR,"{'name': 'Bulgaria', 'country': 'Bulgaria', 'a...","{'type': 'Polygon', 'coordinates': [[[22.65715..."


In [ ]:
# export to json to test for formatting of json
merge_df.to_json('../db/map_data.json', orient='records')